In [11]:
!pip install deap
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

In [6]:
df = pd.read_csv('train.csv')
df.head()

,id,origin_host,origin_port,response_host,response_port,flow_duration,forward_packets_per_sec,backward_packets_per_sec,flow_packets_per_sec,down_up_ratio,...,forward_bulk_packets,backward_bulk_packets,forward_bulk_rate,backward_bulk_rate,active,idle,forward_initial_window_size,backward_initial_window_size,forward_last_window_size,traffic
0,CkwI1TlUCRApPfcJl,103.255.15.27,45314,103.255.15.23,42001,0.000090,11125.474801,NaN,22250.949602,1.00,...,0.0,0.0,NaN,0.0,89.883804,NaN,29200.0,NaN,NaN,Background
1,CBlrcc3dvtaHzyV4zj,103.255.15.27,45060,103.255.15.23,42000,0.000033,30174.848921,30174.848921,60349.697842,1.00,...,0.0,0.0,0.0,0.0,33.140182,NaN,29200.0,0.0,NaN,Background
2,CdpSX33u29yjDvnVzi,103.255.15.23,3440,52.45.16.192,443,61.977190,0.322699,0.242025,0.564724,0.75,...,NaN,0.0,NaN,0.0,401089.021138,9.861604e+06,29200.0,NaN,90.0,Background
3,CT23VJ1KsoKeCdWpx2,103.255.15.150,56132,8.8.8.8,53,0.024249,NaN,82.478178,164.956355,1.00,...,NaN,0.0,0.0,0.0,24248.838425,NaN,0.0,NaN,0.0,Benign
4,C6OJU51P50bwNKvnY6,103.255.15.150,43358,8.8.8.8,53,0.027580,NaN,NaN,72.516256,NaN,...,0.0,0.0,NaN,0.0,27580.022812,0.000000e+00,0.0,NaN,0.0,Benign


One-Hot Encoding For Object data type

In [7]:
# Frequency encoding for origin_host
origin_host_frequency = df['origin_host'].value_counts()
df['origin_host'] = df['origin_host'].map(origin_host_frequency)

# Frequency encoding for response_host
response_host_frequency = df['response_host'].value_counts()
df['response_host'] = df['response_host'].map(response_host_frequency)


In [8]:
df_impute = df.drop(columns = ['id', 'traffic'])

In [9]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Perlu diimpor untuk mengaktifkan IterativeImputer
from sklearn.impute import IterativeImputer

# Pilih kolom numerik untuk imputasi
numeric_columns = df_impute.select_dtypes(include=['float64', 'int64']).columns

# Inisialisasi IterativeImputer
imputer = IterativeImputer(random_state=0)

# Melakukan imputasi pada kolom numerik
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

# Penanganan nilai negatif
df[numeric_columns] = df[numeric_columns].clip(lower=0)


In [10]:
df = df.drop(columns = 'id')

In [12]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
# from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.preprocessing import StandardScaler
# Load your dataset
X = df.drop(columns='traffic')  # Replace 'traffic' with your actual feature columns
y = df['traffic']  # Replace 'traffic' with your actual target column

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y_encoded = label_encoder.fit_transform(y) + 1

# Now y_encoded contains the encoded labels, and df['traffic'] still contains the original labels


In [13]:
# Apply Borderline SMOTE to balance the dataset
smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42, kind='borderline-1')  # 'kind' can be 'borderline-1' or 'borderline-2'
X_balanced, y_balanced = smote.fit_resample(X, y_encoded)

In [14]:
# Convert to DataFrame
df_balanced = pd.DataFrame(X_balanced, columns=X.columns)
df_balanced['traffic'] = y_balanced

# Define number of classes and samples per class
num_classes = len(np.unique(y_encoded))
samples_per_class = 300000 // num_classes

# Sample equally from each class
df_subset = pd.DataFrame()

for label in np.unique(y_encoded):
    df_class = df_balanced[df_balanced['traffic'] == label]
    df_class_sampled = df_class.sample(n=samples_per_class, random_state=42)
    df_subset = pd.concat([df_subset, df_class_sampled])

# Separate features and target
X_balanced_subset = df_subset.drop(columns='traffic')
y_balanced_subset = df_subset['traffic']

In [15]:
# Standardize the features
scaler = StandardScaler()
X_balanced_subset = scaler.fit_transform(X_balanced_subset)

In [16]:
# Split the balanced dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced_subset, y_balanced_subset, test_size=0.2, random_state=42)

In [17]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    activation='relu',  # Logistic activation (sigmoid) in hidden layers
    solver='adam',          # Optimizer, 'adam' is commonly used
    alpha=0.01,             # Ridge penalty factor
    batch_size=400,         # Number of instances for batch processing
    learning_rate_init=0.001, # Initial learning rate
    max_iter=5000,           # Maximum number of iterations
    tol=1e-6,               # Tolerance for the optimization
    hidden_layer_sizes=(150, 100, 50), # Number of hidden units (2 in your case)
    random_state=42,
    verbose=True
)


In [18]:
    # Train the model
mlp.fit(X_train, y_train)


Iteration 1, loss = 0.36877461
Iteration 2, loss = 0.29001705
Iteration 3, loss = 0.27863411
Iteration 4, loss = 0.27307334
Iteration 5, loss = 0.26804503
Iteration 6, loss = 0.26489005
Iteration 7, loss = 0.26136460
Iteration 8, loss = 0.25920063
Iteration 9, loss = 0.25657773
Iteration 10, loss = 0.25361972
Iteration 11, loss = 0.25303650
Iteration 12, loss = 0.25008033
Iteration 13, loss = 0.24796347
Iteration 14, loss = 0.24633138
Iteration 15, loss = 0.24449334
Iteration 16, loss = 0.24388189
Iteration 17, loss = 0.24259050
Iteration 18, loss = 0.24120543
Iteration 19, loss = 0.24057750
Iteration 20, loss = 0.23956285
Iteration 21, loss = 0.23866767
Iteration 22, loss = 0.23704562
Iteration 23, loss = 0.23590636
Iteration 24, loss = 0.23539136
Iteration 25, loss = 0.23469297
Iteration 26, loss = 0.23398588
Iteration 27, loss = 0.23241698
Iteration 28, loss = 0.23221558
Iteration 29, loss = 0.23084605
Iteration 30, loss = 0.23050341
Iteration 31, loss = 0.22939840
Iteration 32, los

MLPClassifier(alpha=0.01, batch_size=400, hidden_layer_sizes=(150, 100, 50),
              max_iter=5000, random_state=42, tol=1e-06, verbose=True)

In [19]:
import pickle

# Save the trained model to a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(mlp, file)


In [20]:
# Make predictions
y_pred = mlp.predict(X_test)


In [21]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import balanced_accuracy_score


# For classification
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# For regression (if your task was regression instead)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.4f}")


Accuracy: 0.9103
Mean Squared Error: 0.3516
Balanced Accuracy: 0.9100


In [36]:
# Load the new data
new_data = pd.read_csv('test.csv')

In [37]:
# Perform the same frequency encoding on the new data
new_data['origin_host'] = new_data['origin_host'].map(origin_host_frequency)
new_data['response_host'] = new_data['response_host'].map(response_host_frequency)

In [38]:
# Drop 'id' and 'traffic' columns before imputation (if present in new data)
new_data_impute = new_data.drop(columns=['id'])

In [39]:
# Impute missing values with the same imputer used on training data
new_data[numeric_columns] = imputer.transform(new_data_impute[numeric_columns])

# Clip negative values to 0
new_data[numeric_columns] = new_data[numeric_columns].clip(lower=0)



In [40]:
new_data_scaled = scaler.transform(new_data.drop(columns=['id']))

In [41]:
# Ubah kembali ke DataFrame untuk penyimpanan
new_data_df = pd.DataFrame(new_data_scaled, columns=new_data.columns[1:])
new_data_df['id'] = new_data['id']

In [42]:
new_data_df.to_csv('data_test.csv', index=False)
